In [ ]:
!pip install transformers Korpora

     |████████████████████████████████| 2.1MB 8.2MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 901kB 37.2MB/s 
     |████████████████████████████████| 3.3MB 53.6MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
ERROR: korpora 0.2.0 has requirement tqdm>=4.46.0, but you'll have tqdm 4.41.1 which is incompatible.
  Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import pandas as pd
import random

In [ ]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name='korean_hate_speech',
    root_dir="/root/Korpora",
    force_download=True,
)

[korean hate speech] download unlabeled_comments_1.txt: 57.0MB [00:00, 126MB/s]                             
[korean hate speech] download unlabeled_comments_2.txt: 59.3MB [00:00, 128MB/s]                             
[korean hate speech] download unlabeled_comments_3.txt: 58.9MB [00:00, 133MB/s]                             
[korean hate speech] download unlabeled_comments_4.txt: 58.2MB [00:00, 125MB/s]                             
[korean hate speech] download unlabeled_comments_5.txt: 3.57MB [00:00, 6.68MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_1.txt: 44.7MB [00:00, 99.4MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_2.txt: 47.4MB [00:00, 76.8MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_3.txt: 48.5MB [00:00, 114MB/s]                             
[korean hate speech] download unlabeled_comments.news_title_4.txt: 47.9MB [00:01, 44.3MB/s]    

In [ ]:
data = pd.read_csv("/root/Korpora/korean_hate_speech/labeled/train.tsv",sep='\t')
# gender_bias = data[['comments','contain_gender_bias']]
# gender_bias

In [ ]:
data

,comments,contain_gender_bias,bias,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate
...,...,...,...,...
7891,힘내세요~ 응원합니다!!,False,none,none
7892,힘내세요~~삼가 고인의 명복을 빕니다..,False,none,none
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,False,none,none
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,False,none,none


In [ ]:
def filtering(x):
    if x=='none': return 0
    else: return 1

offensive = pd.concat([data['comments'], data['hate'].map(filtering)], axis=1)
offensive

,comments,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,1
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",1
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1
...,...,...
7891,힘내세요~ 응원합니다!!,0
7892,힘내세요~~삼가 고인의 명복을 빕니다..,0
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,0
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# gb_train, gb_test, gb_train_label, gb_test_label = train_test_split(gender_bias['comments'], gender_bias['contain_gender_bias'])
b_train, b_test, b_train_label, b_test_label = train_test_split(offensive['comments'], offensive['hate'])

In [ ]:
b_train = list(b_train.values)
b_test = list(b_test.values)
b_train_label = list(b_train_label.values)
b_test_label = list(b_test_label.values)

In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
class BDataSet(Dataset):
    def __init__(self, source, target, tokenizer, input_length, output_length):         
        self.source=source
        self.target = target
        self.input_length = input_length
        self.tokenizer = tokenizer
        self.output_length = output_length
  
    def __len__(self):
        return len(self.target)
    
    # def clean_text(self, text):
    #     text = text.replace('~', '')
    #     return text
      
    def __getitem__(self, index):
        source_i = self.tokenizer.encode_plus(self.source[index], max_length=self.output_length, 
                                                padding='max_length', truncation=True, return_tensors="pt")
        # target_i = torch.tensor([self.target[index][0]])

        target_i = torch.tensor([self.target[index]])
        source_ids = source_i["input_ids"].squeeze()
        src_mask    = source_i["attention_mask"].squeeze()

        return {"input_ids": source_ids, "attention_mask": src_mask, "labels": target_i}

In [ ]:
from transformers import ElectraTokenizer
#, BartTokenizer, GPT2Tokenizer, BertTokenizer, 

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
# tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2")
# tokenizer = BartTokenizer.from_pretrained("hyunwoongko/kobart")
train_dataset = BDataSet(b_train,b_train_label, tokenizer, 16, 16)
test_dataset = BDataSet(b_test,b_test_label, tokenizer, 16, 16)

In [ ]:
# for i in range(len(train_dataset)):
#     # print(tokenizer.decode(train_dataset[i]['input_ids']))
#     print(train_dataset[i])

In [ ]:
from transformers import ElectraForSequenceClassification
# AlbertForSequenceClassification, BertForSequenceClassification, DistilBertForSequenceClassification, BartForSequenceClassification, GPT2ForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
def choose_model(model_name):
    if model_name=='electra': return ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator")
    elif model_name=='distil': return DistilBertForSequenceClassification.from_pretrained("monologg/distilkobert")
    elif model_name=='hanbert': return BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/HanBert-54kN-torch')
    elif model_name=='kobert': return BertForSequenceClassification.from_pretrained("monologg/kobert")
    elif model_name=='kcbert': return BertForSequenceClassification.from_pretrained("beomi/kcbert-base")
    elif model_name=='bert': return BertForSequenceClassification.from_pretrained("bert-base-uncased")
    elif model_name=='bart': return BartForSequenceClassification.from_pretrained("hyunwoongko/kobart", return_dict=True)
    elif model_name=='albert': return AlbertForSequenceClassification.from_pretrained("kykim/albert-kor-base")
    elif model_name=='gpt2': return GPT2ForSequenceClassification.from_pretrained("skt/kogpt2-base-v2")
    else: 
        print("model not in list")
        return 0

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to('cuda')

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=8,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    # learning_rate= 1e-4,
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
200,0.652300,0.592825,0.708713,4.457500,442.848000
400,0.579500,0.553021,0.736575,4.526400,436.105000
600,0.506600,0.528330,0.738095,4.570900,431.867000
800,0.444500,0.715536,0.740628,4.518700,436.851000
1000,0.326100,0.591540,0.708207,4.428600,445.743000
1200,0.262100,1.054329,0.716312,4.473100,441.301000
1400,0.200900,0.941070,0.714792,4.440700,444.524000
1600,0.159000,1.082834,0.731003,4.474600,441.153000
1800,0.126500,1.183518,0.723404,4.444800,444.119000
2000,0.075000,1.357820,0.725937,4.503100,438.365000


TrainOutput(global_step=2968, training_loss=0.23699268891483627, metrics={'train_runtime': 662.2888, 'train_samples_per_second': 4.481, 'total_flos': 513584107932672.0, 'epoch': 8.0, 'init_mem_cpu_alloc_delta': 4096, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -76107776, 'train_mem_gpu_alloc_delta': 1813327872, 'train_mem_cpu_peaked_delta': 88645632, 'train_mem_gpu_peaked_delta': 0})

In [ ]:
trainer.evaluate()
# evaluate the current model after training

{'epoch': 8.0,
 'eval_accuracy': 0.7380952380952381,
 'eval_loss': 0.528329610824585,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 12231680,
 'eval_runtime': 4.4387,
 'eval_samples_per_second': 444.723}

In [ ]:
# gdrive에 mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# saving the fine tuned model & tokenizer
model_path = "/content/gdrive/MyDrive/offensive-koelectra"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/gdrive/MyDrive/offensive-koelectra/tokenizer_config.json',
 '/content/gdrive/MyDrive/offensive-koelectra/special_tokens_map.json',
 '/content/gdrive/MyDrive/offensive-koelectra/vocab.txt',
 '/content/gdrive/MyDrive/offensive-koelectra/added_tokens.json')

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=216, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax()

In [ ]:
# Example #1
text = "여자가 그러면 안되지;;"
print(get_prediction(text))

tensor(1, device='cuda:0')
